# OpenCV Basics
This notebook introduces the core OpenCV workflow: loading images, inspecting pixels, converting color spaces, and grabbing frames from a camera.

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

## Load and Display an Image
Update `image_path` to point at any local image. If OpenCV cannot find the file we will generate a simple placeholder so the rest of the notebook continues to run.

In [ ]:
image_path = "../images/nl_clown.jpg"
image = cv2.imread(image_path)
if image is None:
    placeholder = np.zeros((256, 256, 3), dtype=np.uint8)
    cv2.putText(placeholder, "No image", (30, 130), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 255), 2, cv2.LINE_AA)
    image = placeholder

plt.figure(figsize=(4, 4))
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
plt.title("Loaded image (RGB)")
plt.axis("off")

### Question
What do the four values mean, that are printed above the image?

## Inspect Pixels
OpenCV stores color images in BGR order by default. The snippet below prints a sample pixel so you can verify the channel ordering and numeric range.

In [ ]:
height, width = image.shape[:2]
print(f"Image size: {width}x{height}")
center = (height // 2, width // 2)
print("Center pixel (B, G, R):", image[center])

### Question
Why are the colors in BGR order instead of RGB?

## Color Conversions
Convert the input to grayscale.

In [ ]:
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

fig, axes = plt.subplots(1, 2, figsize=(9, 4))
axes[0].imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
axes[0].set_title("RGB")
axes[0].axis("off")
axes[1].imshow(gray, cmap="gray")
axes[1].set_title("Grayscale")
axes[1].axis("off")
plt.tight_layout()

## Color spaces
Using the HSV color space can make it easier to segment colors based on human perception. The HSV color space separates color information (hue) from intensity (value), allowing for more intuitive color-based segmentation.

In [ ]:
hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
hue = hsv[:, :, 0]
saturation = hsv[:, :, 1]
value = hsv[:, :, 2]

fig, axes = plt.subplots(1, 4, figsize=(16, 4))
axes[0].imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
axes[0].set_title("RGB")
axes[0].axis("off")
axes[1].imshow(hue, cmap="hsv")
axes[1].set_title("Hue")
axes[1].axis("off")
axes[2].imshow(cv2.cvtColor(saturation, cv2.COLOR_GRAY2BGR))
axes[2].set_title("Saturation")
axes[2].axis("off")
axes[3].imshow(cv2.cvtColor(value, cv2.COLOR_GRAY2BGR))
axes[3].set_title("Value")
axes[3].axis("off")
plt.tight_layout()

### Question
Why does the hue channel image looks so distorted for the forehead, eyes and lips of the clown?

## Save Processed Results
Saving results is useful when building pipelines. This cell writes the grayscale image beside the original.

In [ ]:
output_path = "../images/sample_gray.jpg"
cv2.imwrite(output_path, gray)
print(f"Saved grayscale copy to {output_path}")

## Capture from a Camera
This example shows how to capture a single frame from the default camera. It safely releases the camera even if no frame is read.

In [ ]:
from IPython.display import display, Image, clear_output
import time

# Capture and display live frames from the default webcam inline in Jupyter

cap = cv2.VideoCapture(0)
if not cap.isOpened():
    raise RuntimeError("Cannot open webcam (VideoCapture(0) failed)")

try:
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Failed to grab frame")
            break
        # Encode as JPEG and display inline (fast and avoids creating new matplotlib figures)
        _, buf = cv2.imencode('.jpg', frame)
        display(Image(data=buf.tobytes()))
        clear_output(wait=True)
        time.sleep(0.03)  # ~30 FPS-ish; adjust as needed
except KeyboardInterrupt:
    # Stop the loop with Ctrl+C in the notebook
    pass
finally:
    cap.release()
    clear_output()
    print("Webcam stopped and released.")

## Task
Using the code snippets above, create a new cell that displays the webcam image but the colors are fully saturated.